In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
MEA = '102016_MEAC'

pathtimestamps = '/Volumes/MEA_DATA_2/102016_MEAC/numpy_neurons/combinedtimestamps/beforestim_timestamps/Circadian3day/'
timestamps = os.listdir(pathtimestamps) # lists the directory containing all the timestamps from that MEA
timestamps = np.sort(timestamps) # so that folder ends up at the end of the list

pathtosave = '/Volumes/MEA_DATA_2/102016_MEAC/numpy_neurons/combinedtimestamps/beforestim_timestamps/3day_results/'

if os.path.isdir(pathtosave) == False:
    os.mkdir(pathtosave)
    

In [2]:
for i in range(len(timestamps)):
    if timestamps[i][0:11] == 'PeriodPhase':
        periodphase = np.genfromtxt(pathtimestamps+timestamps[i], delimiter = ',')

In [7]:
#Calculate Maximums and Print out max and graphs

x = 0
maxvalue = []

for i in range(len(timestamps)):
    if timestamps[i][-3:] == 'npy':
        pathfile = pathtimestamps + timestamps[i]
        neuron = np.load(pathfile)
        neuron = np.sort(neuron)

        counts, bins = np.histogram(neuron, int(neuron[-1]/600)) # calculates histogram in 10min bins
        counts = np.float_(counts)/600
        if x == 10000:
            jtkphase = periodphase[i-1,1]*6
            jtkperiod = periodphase[i-1,0]*6
        else:    
            jtkphase = periodphase[i,1]*6
            jtkperiod = periodphase[i,0]*6
        
        startpoint = jtkphase - (jtkperiod/2)
        if np.isnan(jtkphase):
            print 'ERROR ' + timestamps[i]
            continue

        if len(counts) <= jtkperiod:
            print 'ERROR '+timestamps[i]
            continue

        if startpoint > 0:
            startpoint = startpoint
        else:
            startpoint = jtkphase + (jtkperiod/2)

        day1start = startpoint
        day2start = startpoint + jtkperiod
        day3start = day2start + jtkperiod
        day3end = day3start + jtkperiod

        day1max = max(counts[day1start:day2start])
        day1maxtime = np.argmax(counts[day1start:day2start])
        day1min = min(counts[day1start:day2start])
        day1mintime = np.argmin(counts[day1start:day2start])
        firingtotalday1 = np.sum(counts[day1start:day2start]*600)

        if day3start < len(counts):
            day2max = max(counts[day2start:day3start])
            day2maxtime = np.argmax(counts[day2start:day3start])
            day2min = min(counts[day2start:day3start])
            day2mintime = np.argmin(counts[day2start:day3start])
            firingtotalday2 = np.sum(counts[day2start:day3start]*600)
            aftermax1 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day2start > jtkperiod/2:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = max(counts[day2start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax1time = np.argmax(counts[day2start:len(counts)])
        else:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = 'NaN'
            aftermin = 'NaN'

        if day3end < len(counts):
            day3max = max(counts[day3start:day3end])
            day3maxtime = np.argmax(counts[day3start:day3end])
            day3min = min(counts[day3start:day3end])
            day3mintime = np.argmin(counts[day3start:day3end])
            firingtotalday3 = np.sum(counts[day3start:day3end]*600)
            aftermax2 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day3start > jtkperiod/2:
            aftermax2 = max(counts[day3start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax2time = np.argmax(counts[day3start:len(counts)])
            day3max = 'NaN'
            day3min = 'NaN'
        else:
            day3max = 'NaN'
            day3min = 'NaN'
            aftermax2 = 'NaN'
            aftermin = 'NaN'

        if day1start > jtkperiod/2:
            beforemax = max(counts[0:day1start])
            beforemin = min(counts[0:day1start])
            beforemaxtime = np.argmax(counts[0:day1start])
        else:
            beforemax = 'NaN'
            beforemin = 'NaN'

        maxall = [timestamps[i], day1max]
        if day2max != 'NaN':
            maxall.append(day2max)
        else:
            maxall.append(' ')
        if day3max != 'NaN':
            maxall.append(day3max)
        else: 
            maxall.append(' ')
        if beforemax != 'NaN':
            maxall.append(beforemax)
        else:
            maxall.append(' ')
        if aftermax1 != 'NaN':
            maxall.append(aftermax1)
        else:
            maxall.append(' ')
        if aftermax2 != 'NaN':
            maxall.append(aftermax2)
        else:
            maxall.append(' ')
        
        maxvalue.append([maxall])
        print maxall
        
        
        #create rate histogram plot demarcating circadian days, max, min
        plt.close()
        plt.figure()
        plt.plot(counts)
        ax = plt.gca()
        ax.set_xticks([day1start,day2start,day3start,day3end], minor=False)
        ax.xaxis.grid(True, which='major', linestyle = '--')
        ax.plot([jtkphase], [counts[jtkphase]], '*', color = 'red', markersize = 20)
        ax.annotate('MaxDay1', xy = (day1maxtime+day1start, day1max), xytext=(day1maxtime+day1start, day1max+.25), arrowprops=dict(facecolor='black', width =1, headwidth = 5, shrink =0.01))
        ax.annotate('MinDay1', xy = (day1mintime+day1start, day1min), xytext=(day1mintime+day1start, day1min+.25), arrowprops=dict(facecolor='black', width =1, shrink = .01, headwidth = 5))
        
        if day2max != 'NaN':
            ax.annotate('MaxDay2', xy = (day2maxtime+day2start, day2max), xytext=(day2maxtime+day2start, day2max+.25), arrowprops=dict(facecolor='black', width =1, headwidth = 5, shrink =0.01))
            ax.annotate('MinDay2', xy = (day2mintime+day2start, day2min), xytext=(day2mintime+day2start, day2min+.25), arrowprops=dict(facecolor='black', width =1, headwidth = 5, shrink =0.01))
        if day3max != 'NaN':
            ax.annotate('MaxDay3', xy = (day3maxtime+day3start, day3max), xytext=(day3maxtime+day3start, day3max+.25), arrowprops=dict(facecolor='black', width =1, headwidth = 5, shrink =0.01))
            ax.annotate('MinDay3', xy = (day3mintime+day3start, day3min), xytext=(day3mintime+day3start, day3min+.25), arrowprops=dict(facecolor='black', width =1, headwidth = 5, shrink =0.01))
        if beforemax != 'NaN':
            ax.annotate('BeforeMax', xy = (beforemaxtime, beforemax), xytext=(beforemaxtime, beforemax+.25), arrowprops=dict(facecolor='black', width = 1, headwidth = 5, shrink =0.01))
        if aftermax1 != 'NaN':
            ax.annotate('AfterMax', xy = (aftermax1time+day2start, aftermax1), xytext=(aftermax1time+day2start, aftermax1+.25), arrowprops = dict(facecolor='black', width =1, headwidth = 5, shrink = 0.01))
        if aftermax2 != 'NaN':
            ax.annotate('AfterMax', xy = (aftermax2time+day3start, aftermax2), xytext=(aftermax2time+day3start, aftermax2+.25), arrowprops=dict(facecolor='black',width=1,headwidth=5,shrink=0.01))
        
        plt.savefig(pathtosave+timestamps[i][0:12])
    else:
        x = 10000
        continue
        
np.savetxt(pathtosave+'maxvalues.csv', maxvalue, delimiter = ',', fmt='%s')
print 'finished!'

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:41: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:43: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-i

['14_neuron3_before.npy', 3.1916666666666669, 3.0866666666666664, ' ', ' ', ' ', 3.0899999999999999]


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:87: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:88: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:89: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


['16_neuron4_before.npy', 1.4516666666666667, 1.175, ' ', 0.95833333333333337, ' ', ' ']


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:57: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:59: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


['25_neuron2_before.npy', 1.0683333333333334, ' ', ' ', 1.325, 0.34666666666666668, ' ']
['25_neuron4_before.npy', 2.2000000000000002, ' ', ' ', 3.4950000000000001, 2.125, ' ']
['26_neuron2_before.npy', 3.5616666666666665, 2.1633333333333336, ' ', 4.125, ' ', ' ']
['32_neuron17and32_neuron9_.npy', 1.7649999999999999, 1.51, ' ', ' ', ' ', ' ']
['32_neuron1_before.npy', 3.25, 4.003333333333333, ' ', ' ', ' ', 2.4466666666666668]
['33_neuron1_before.npy', 1.7716666666666667, 1.7783333333333333, ' ', 2.1566666666666667, ' ', ' ']
['33_neuron5_before.npy', 3.6583333333333332, 2.8816666666666668, ' ', 1.9883333333333333, ' ', ' ']
['33_neuron6_before.npy', 1.5649999999999999, 2.0216666666666665, ' ', ' ', ' ', ' ']
['33_neuron7_before.npy', 0.50666666666666671, 2.5066666666666668, ' ', ' ', ' ', ' ']
['35_neuron1_before.npy', 0.625, 1.1766666666666667, ' ', ' ', ' ', 0.72833333333333339]
['35_neuron7_before.npy', 2.4416666666666669, 2.3966666666666665, ' ', ' ', ' ', ' ']
['37_neuron2_before

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:69: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:70: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:71: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


['52_neuron7_before.npy', 0.89166666666666672, 1.1899999999999999, 1.2066666666666668, ' ', ' ', ' ']
['53_neuron4_before.npy', 3.0550000000000002, 3.1566666666666667, ' ', 2.1966666666666668, ' ', ' ']
['53_neuron5_before.npy', 0.82333333333333336, 0.7416666666666667, ' ', 1.3200000000000001, ' ', ' ']
['54_neuron10before.npy', 0.16833333333333333, 0.27333333333333332, ' ', 3.2799999999999998, ' ', ' ']
['54_neuron4_before.npy', 0.68166666666666664, 2.0649999999999999, 3.8250000000000002, ' ', ' ', ' ']
['54_neuron7_before.npy', 1.5783333333333334, 1.5449999999999999, ' ', 0.22166666666666668, ' ', ' ']
['62_neuron2_before.npy', 1.7266666666666666, 1.1983333333333333, ' ', 0.24666666666666667, ' ', ' ']
['62_neuron4_before.npy', 1.0733333333333333, 0.97999999999999998, ' ', 1.9666666666666666, ' ', 0.95333333333333337]
['62_neuron7_before.npy', 1.345, 1.2416666666666667, ' ', ' ', ' ', 1.8666666666666667]
['62_neuron8_before.npy', 0.28833333333333333, 0.14000000000000001, 2.0016666666

In [9]:
#Print out Minimums

x = 0
for i in range(len(timestamps)-1):
    
    if timestamps[i] == '.DS_Store':
        x = 10000
        continue
    else:
        pathfile = pathtimestamps + timestamps[i]
        neuron = np.genfromtxt(pathfile, delimiter = ',') #imports specified neuron
        neuron = np.sort(neuron)

        periodphase = np.genfromtxt(pathtimestamps2, delimiter = ',') #imports period/phase values for entire MEA
        # starts at index 1 - period is i,0 JTK cycle phase is i,1 Both are in h

        # plan is to use JTK phase marker and then subtract half of the JTK period estimate
        # if that # is negative it means that the cycle is not intact - cannot calculate values
        # start with next cycle - in other words phase marker plus half the JTK period estimate

        counts, bins = np.histogram(neuron, int(neuron[-1]/600)) # calculates histogram in 10min bins
        counts = np.float_(counts)/600
        if x == 10000:
            jtkphase = periodphase[i,1]*6
            jtkperiod = periodphase[i,0]*6
        else:    
            jtkphase = periodphase[i+1,1]*6
            jtkperiod = periodphase[i+1,0]*6
        
        startpoint = jtkphase - (jtkperiod/2)
        if np.isnan(jtkphase):
            print 'ERROR ' + timestamps[i]
            continue

        if len(counts) <= jtkperiod:
            print 'ERROR '+timestamps[i]
            continue

        if startpoint > 0:
            startpoint = startpoint
        else:
            startpoint = jtkphase + (jtkperiod/2)


        day1start = startpoint
        day2start = startpoint + jtkperiod
        day3start = day2start + jtkperiod
        day3end = day3start + jtkperiod

        day1max = max(counts[day1start:day2start])
        day1maxtime = np.argmax(counts[day1start:day2start])
        day1min = min(counts[day1start:day2start])
        day1mintime = np.argmin(counts[day1start:day2start])
        firingtotalday1 = np.sum(counts[day1start:day2start]*600)

        if day3start < len(counts):
            day2max = max(counts[day2start:day3start])
            day2maxtime = np.argmax(counts[day2start:day3start])
            day2min = min(counts[day2start:day3start])
            day2mintime = np.argmin(counts[day2start:day3start])
            firingtotalday2 = np.sum(counts[day2start:day3start]*600)
            aftermax1 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day2start > jtkperiod/2:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = max(counts[day2start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax1time = np.argmax(counts[day2start:len(counts)])
        else:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = 'NaN'
            aftermin = 'NaN'

        if day3end < len(counts):
            day3max = max(counts[day3start:day3end])
            day3maxtime = np.argmax(counts[day3start:day3end])
            day3min = min(counts[day3start:day3end])
            day3mintime = np.argmin(counts[day3start:day3end])
            firingtotalday3 = np.sum(counts[day3start:day3end]*600)
            aftermax2 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day3start > jtkperiod/2:
            aftermax2 = max(counts[day3start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax2time = np.argmax(counts[day3start:len(counts)])
            day3max = 'NaN'
            day3min = 'NaN'
        else:
            day3max = 'NaN'
            day3min = 'NaN'
            aftermax2 = 'NaN'
            aftermin = 'NaN'

        if day1start > jtkperiod/2:
            beforemax = max(counts[0:day1start])
            beforemin = min(counts[0:day1start])
            beforemaxtime = np.argmax(counts[0:day1start])
        else:
            beforemax = 'NaN'
            beforemin = 'NaN'

        minall = [day1min]
        if day2min != 'NaN':
            minall.append(day2min)
        if day3min != 'NaN':
            minall.append(day3min)

        print minall

print 'finished!'

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:57: VisibleDeprecationWarning: using a non-i

[0.0, 0.0]


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:97: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:98: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[0.0, 0.0]


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:69: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[0.0]
[0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.063333333333333339, 0.093333333333333338]
[0.055, 0.105]
[0.0083333333333333332, 0.02]
[0.036666666666666667, 0.11333333333333333]
[0.016666666666666666, 0.081666666666666665]
[0.0016666666666666668, 0.0016666666666666668]
[0.0]
[0.0016666666666666668]
[0.0033333333333333335, 0.0016666666666666668]
[0.0016666666666666668, 0.0016666666666666668]
finished!


In [10]:
#Print out Firing Totals

x = 0
for i in range(len(timestamps)-1):
    
    if timestamps[i] == '.DS_Store':
        x = 10000
        continue
    else:
        pathfile = pathtimestamps + timestamps[i]
        neuron = np.genfromtxt(pathfile, delimiter = ',') #imports specified neuron
        neuron = np.sort(neuron)

        periodphase = np.genfromtxt(pathtimestamps2, delimiter = ',') #imports period/phase values for entire MEA
        # starts at index 1 - period is i,0 JTK cycle phase is i,1 Both are in h

        # plan is to use JTK phase marker and then subtract half of the JTK period estimate
        # if that # is negative it means that the cycle is not intact - cannot calculate values
        # start with next cycle - in other words phase marker plus half the JTK period estimate

        counts, bins = np.histogram(neuron, int(neuron[-1]/600)) # calculates histogram in 10min bins
        counts = np.float_(counts)/600
        if x == 10000:
            jtkphase = periodphase[i,1]*6
            jtkperiod = periodphase[i,0]*6
        else:    
            jtkphase = periodphase[i+1,1]*6
            jtkperiod = periodphase[i+1,0]*6
        
        startpoint = jtkphase - (jtkperiod/2)
        if np.isnan(jtkphase):
            print 'ERROR ' + timestamps[i]
            continue

        if len(counts) <= jtkperiod:
            print 'ERROR '+timestamps[i]
            continue

        if startpoint > 0:
            startpoint = startpoint
        else:
            startpoint = jtkphase + (jtkperiod/2)


        day1start = startpoint
        day2start = startpoint + jtkperiod
        day3start = day2start + jtkperiod
        day3end = day3start + jtkperiod

        day1max = max(counts[day1start:day2start])
        day1maxtime = np.argmax(counts[day1start:day2start])
        day1min = min(counts[day1start:day2start])
        day1mintime = np.argmin(counts[day1start:day2start])
        firingtotalday1 = np.sum(counts[day1start:day2start]*600)

        if day3start < len(counts):
            day2max = max(counts[day2start:day3start])
            day2maxtime = np.argmax(counts[day2start:day3start])
            day2min = min(counts[day2start:day3start])
            day2mintime = np.argmin(counts[day2start:day3start])
            firingtotalday2 = np.sum(counts[day2start:day3start]*600)
            aftermax1 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day2start > jtkperiod/2:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = max(counts[day2start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax1time = np.argmax(counts[day2start:len(counts)])
        else:
            day2max = 'NaN'
            day2min = 'NaN'
            aftermax1 = 'NaN'
            aftermin = 'NaN'

        if day3end < len(counts):
            day3max = max(counts[day3start:day3end])
            day3maxtime = np.argmax(counts[day3start:day3end])
            day3min = min(counts[day3start:day3end])
            day3mintime = np.argmin(counts[day3start:day3end])
            firingtotalday3 = np.sum(counts[day3start:day3end]*600)
            aftermax2 = 'NaN'
            aftermin = 'NaN'
        elif len(counts) - day3start > jtkperiod/2:
            aftermax2 = max(counts[day3start:len(counts)])
            aftermin = min(counts[day2start:len(counts)])
            aftermax2time = np.argmax(counts[day3start:len(counts)])
            day3max = 'NaN'
            day3min = 'NaN'
        else:
            day3max = 'NaN'
            day3min = 'NaN'
            aftermax2 = 'NaN'
            aftermin = 'NaN'

        if day1start > jtkperiod/2:
            beforemax = max(counts[0:day1start])
            beforemin = min(counts[0:day1start])
            beforemaxtime = np.argmax(counts[0:day1start])
        else:
            beforemax = 'NaN'
            beforemin = 'NaN'

        firingall = [firingtotalday1]
        if day2min != 'NaN':
            firingall.append(firingtotalday2)
        if day3min != 'NaN':
            firingall.append(firingtotalday3)

        print firingall

print 'finished!'

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:57: VisibleDeprecationWarning: using a non-i

[21912.0, 52306.0]


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:97: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:98: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:69: VisibleDeprecationWarning: using a non-i

[29559.0, 36734.0]
[39933.0]
[35693.0]
[10777.0, 155205.0]
[62154.0, 8835.0]
[116658.0, 108914.0]
[5333.0, 66956.0]
[17990.0, 21437.0]
[28290.0, 46581.0]
[88009.0, 106038.0]
[72403.0, 128341.0]
[33724.0, 30225.0]
[16062.0, 30252.0]
[27916.0, 44884.0]
[10976.0]
[17023.0]
[16791.0, 34899.0]
[22200.0, 67561.0]
finished!


In [11]:
p = 0
for i in range(len(timestamps)-1):
    if timestamps[i] == '.DS_Store':
        p = 10000
        continue
    else:
        pathfile = pathtimestamps + timestamps[i]
        neuron = np.genfromtxt(pathfile, delimiter = ',') #imports specified neuron
        neuron = np.sort(neuron)

        periodphase = np.genfromtxt(pathtimestamps2, delimiter = ',') #imports period/phase values for entire MEA
        # starts at index 1 - period is i,0 JTK cycle phase is i,1 Both are in h

        # plan is to use JTK phase marker and then subtract half of the JTK period estimate
        # if that # is negative it means that the cycle is not intact - cannot calculate values
        # start with next cycle - in other words phase marker plus half the JTK period estimate

        counts, bins = np.histogram(neuron, int(neuron[-1]/600)) # calculates histogram in 10min bins
        counts = np.float_(counts)/600

        if p == 10000:
            jtkphase = periodphase[i,1]*6
            jtkperiod = periodphase[i,0]*6
        else:    
            jtkphase = periodphase[i+1,1]*6
            jtkperiod = periodphase[i+1,0]*6
        
        startpoint = jtkphase - (jtkperiod/2)
        if np.isnan(jtkphase):
            print 'ERROR ' + timestamps[i]
            continue

        if len(counts) <= jtkperiod:
            print 'ERROR '+timestamps[i]
            continue

        if startpoint > 0:
            startpoint = startpoint
        else:
            startpoint = jtkphase + (jtkperiod/2)

        day1start = startpoint
        day2start = startpoint + jtkperiod
        day3start = day2start + jtkperiod
        day3end = day3start + jtkperiod

        day1max = max(counts[day1start:day2start])
        day1min = min(counts[day1start:day2start])

        if day3start < len(counts):
            day2max = max(counts[day2start:day3start])
            day2min = min(counts[day2start:day3start])

        else:
            day2max = 'NaN'
            day2min = 'NaN'

        if day3end < len(counts):
            day3max = max(counts[day3start:day3end])
            day3min = min(counts[day3start:day3end])
        else:
            day3max = 'NaN'
            day3min = 'NaN'

        for x in range(10):
            fraction = np.float_(x+1)/np.float_(10)
            day1_fraction = (fraction*(day1max-day1min))+day1min
            if day2max != 'NaN':
                day2_fraction = (fraction*(day2max-day2min))+day2min
            else:
                day2_fraction = ' '
            if day3max != 'NaN':  
                day3_fraction = (fraction*(day3max-day3min))+day3min
            else:
                day3_fraction = ' '

            alpha = []
            alpha2 = []
            alpha3 = []

            for j in range(int(jtkperiod)):
                if np.float((counts[j+day1start-1])) > day1_fraction:
                    alpha = np.append(alpha, 1)
                if day2max != 'NaN':
                    if np.float((counts[j+day2start-1])) > day2_fraction:
                        alpha2 = np.append(alpha2,1)
                if day3max != 'NaN':
                    if np.float((counts[j+day3start-1])) > day3_fraction:
                        alpha3 = np.append(alpha3,1)

            sumalpha1 = np.sum(alpha)
            sumalpha1h = np.float_(sumalpha1)/6
            if day2max != 'NaN':
                sumalpha2 = np.sum(alpha2)
                sumalpha2h = np.float_(sumalpha2)/6
            else:
                sumalpha2 = ' '
                sumalpha2h = ' '

            if day3max != 'NaN':
                sumalpha3 = np.sum(alpha3)
                sumalpha3h = np.float_(sumalpha3)/6
            else:
                sumalpha3 = ' '
                sumalpha3h = ' '

            sumalphatotal = [fraction, sumalpha1h, sumalpha2h, sumalpha3h]
            print sumalphatotal
print 'finished!'

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:48: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:82: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:85: VisibleDeprecationWarning: using a non-i

[0.10000000000000001, 10.833333333333334, 10.333333333333334, ' ']
[0.20000000000000001, 8.0, 10.0, ' ']
[0.29999999999999999, 4.666666666666667, 9.5, ' ']
[0.40000000000000002, 3.3333333333333335, 8.5, ' ']
[0.5, 2.6666666666666665, 7.5, ' ']
[0.59999999999999998, 2.0, 7.0, ' ']
[0.69999999999999996, 1.1666666666666667, 5.666666666666667, ' ']
[0.80000000000000004, 0.5, 3.8333333333333335, ' ']
[0.90000000000000002, 0.5, 0.66666666666666663, ' ']
[1.0, 0.0, 0.0, ' ']
[0.10000000000000001, 10.833333333333334, 8.8333333333333339, ' ']
[0.20000000000000001, 10.0, 8.3333333333333339, ' ']
[0.29999999999999999, 7.333333333333333, 7.666666666666667, ' ']
[0.40000000000000002, 6.666666666666667, 7.333333333333333, ' ']
[0.5, 6.5, 5.833333333333333, ' ']
[0.59999999999999998, 5.333333333333333, 5.666666666666667, ' ']
[0.69999999999999996, 3.1666666666666665, 4.5, ' ']
[0.80000000000000004, 2.5, 3.0, ' ']
[0.90000000000000002, 1.0, 1.5, ' ']
[1.0, 0.0, 0.0, ' ']
[0.10000000000000001, 3.0, ' '

In [ ]:
x